In [146]:
#importing all the required libraries
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
import nltk 
import random 

from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [147]:
#importing dataset
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('/content/train.tsv',delimiter='\t',encoding='utf-8')
df.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [148]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,22044,989,"the director , Tom Dey , had spliced together ...",3
1,148279,8069,rated EEE for excitement,4
2,37860,1800,realistic portrayal,3
3,37115,1759,on the amateurish,1
4,100473,5271,hardest,3


In [149]:
#Spliting the data into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=2003)
docs=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())
for i in range(len(X_train)):
  docs.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  docs.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

docs[0]

[['clamorous'], 2]

In [0]:
#Data handling 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"

remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True


In [151]:
#prepocessing ideas for Textual Feature Extraction
for l in range(len(docs)):                   
  label = docs[l][1]                         
  tmpReview = []                                  
  for w in docs[l][0]:                        
    newWord = w                                   
    if remove_stopwords and (w in stopwords_en):  
      continue                                     
    if removePuncs and (w in punctuations):       
      continue                                   
    if useStemming:
      newWord = lancaster.stem(newWord)
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                     
  docs[l] = (tmpReview, label)              
  docs[l] = (' '.join(tmpReview), label) 

print(docs[0])

('clamorous', 2)


In [152]:
df = pd.DataFrame(docs, columns=['text', 'sentiment']) 
df.head()

,text,sentiment
0,clamorous,2
1,appreciated anyone outside under-10 set,1
2,A poignant comedy,3
3,trust,2
4,motivated something franchise possibilities,3


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

In [0]:
#vectorization
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2000) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test


In [155]:
Y_test

13510     3
61932     1
82549     2
137718    2
121990    2
         ..
94224     3
135456    2
154729    1
23031     2
57870     4
Name: sentiment, Length: 46818, dtype: int64

In [0]:
#Importing all the required libraries for the keras model
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:
batch_size = 64
num_classes = 5
epochs = 12

In [158]:
X_train.shape
# Y_train = to_categorical(Y_train,5)
# Y_test = to_categorical(Y_test,5)

(109242, 2000)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)


In [160]:
Y_test

array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [0]:
#Applying different layers 
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,
                 activation='relu',
                 input_shape=(2000,1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(rate = 0.50))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [164]:
model.fit(X_train, Y_train,batch_size=64,epochs=12)

score = model.evaluate(X_test, Y_test, verbose=0)
print('Testing - loss:', score[0])
print('Testing - accuracy:', score[1])

Epoch 1/12
109242/109242 [==============================] - 100s 919us/step - loss: 1.1404 - acc: 0.5551
Epoch 2/12
109242/109242 [==============================] - 99s 908us/step - loss: 1.0547 - acc: 0.5918
Epoch 3/12
109242/109242 [==============================] - 102s 936us/step - loss: 1.0332 - acc: 0.6018
Epoch 4/12
109242/109242 [==============================] - 99s 909us/step - loss: 1.0260 - acc: 0.6054
Epoch 5/12
109242/109242 [==============================] - 98s 899us/step - loss: 1.0196 - acc: 0.6082
Epoch 6/12
109242/109242 [==============================] - 100s 916us/step - loss: 1.0151 - acc: 0.6096
Epoch 7/12
109242/109242 [==============================] - 101s 920us/step - loss: 1.0124 - acc: 0.6109
Epoch 8/12
109242/109242 [==============================] - 100s 919us/step - loss: 1.0082 - acc: 0.6128
Epoch 9/12
109242/109242 [==============================] - 98s 896us/step - loss: 1.0051 - acc: 0.6159
Epoch 10/12
109242/109242 [==============================] 